In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy as sp
import scipy.stats as stats
import statsmodels.formula.api as smf

In [3]:
hmd = pd.read_csv("../hmd_with_excess_mortality.csv", parse_dates = ["date"])
covid = pd.read_csv("../selected_covid_weekly.csv")

In [4]:
#create a dictionary that translates country into population
pop_dict = covid.groupby('country')['population'].first().to_dict()

In [19]:
hmd

,Unnamed: 0.2,Unnamed: 0.1,date,Unnamed: 0,country,week,year,DTotal,mov_avg,years_relative,pred_deaths
0,0,10501,1990-01-07,10501,FIN,1,1990,1146.0,1146.000000,-30,993.782488
1,1,10533,1990-01-14,10533,FIN,2,1990,1139.0,1142.500000,-30,1009.888018
2,2,10565,1990-01-21,10565,FIN,3,1990,1185.0,1156.666667,-30,1020.113159
3,3,10597,1990-01-28,10597,FIN,4,1990,1164.0,1158.500000,-30,1024.963902
4,4,10629,1990-02-04,10629,FIN,5,1990,1103.0,1147.400000,-30,1022.074409
...,...,...,...,...,...,...,...,...,...,...,...
34634,35415,20609,2022-01-30,20609,LTU,4,2022,917.0,962.000000,2,851.896321
34635,35416,13433,2022-02-06,13433,GBR,5,2022,1610.0,11961.571429,2,12989.842857
34636,35417,7491,2022-02-06,7491,DNK,5,2022,1193.0,1236.428571,2,1067.978022
34637,35418,24108,2022-02-06,24108,NLD,5,2022,3450.0,3384.285714,2,3112.960176


In [25]:
hmd['em_per_capita'] = hmd['excess_mortality'] / hmd['country'].map(pop_dict)

In [30]:
hmd = hmd.drop(['Unnamed: 0.2'], axis = 1)
hmd = hmd.drop(['Unnamed: 0.1'], axis = 1)
hmd = hmd.drop(['Unnamed: 0'], axis = 1)


In [32]:
max_em_per_capita = hmd.em_per_capita.max()
min_em_per_capita = hmd.em_per_capita.min()
print(max_em_per_capita, min_em_per_capita)

0.017982702944009037 -0.0057672814179191845


In [33]:
hmd['normalized_em'] = (hmd['em_per_capita'] - min_em_per_capita) / (max_em_per_capita - min_em_per_capita)

In [48]:
hmd.to_csv('hmd_normalized_em.csv')

In [6]:
covid = covid.drop(['Unnamed: 0'], axis = 1)

In [7]:
covid

,country,year,week,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,hosp_patients,population
0,AUS,2020,4,4.0,4.0,NaN,0.0,0.155,NaN,25788217.0
1,AUS,2020,5,5.0,8.0,NaN,0.0,0.194,NaN,25788217.0
2,AUS,2020,6,12.0,3.0,NaN,0.0,0.465,NaN,25788217.0
3,AUS,2020,7,15.0,0.0,NaN,0.0,0.582,NaN,25788217.0
4,AUS,2020,8,15.0,0.0,NaN,0.0,0.582,NaN,25788217.0
...,...,...,...,...,...,...,...,...,...,...
3651,USA,2022,1,56345834.0,5064970.0,830299.0,11876.0,169249.873,108779.0,332915074.0
3652,USA,2022,2,61703251.0,5614161.0,842288.0,13542.0,185342.316,142248.0,332915074.0
3653,USA,2022,3,66610143.0,4951791.0,854838.0,14705.0,200081.487,150906.0,332915074.0
3654,USA,2022,4,71802940.0,3561887.0,870600.0,16801.0,215679.450,144756.0,332915074.0


In [8]:
covid['mov_avg_cases'] = covid.groupby('country')['new_cases'].transform(lambda x: x.rolling(7, 1).mean())

In [11]:
covid['new_cases_per_capita'] = covid['mov_avg_cases'] / covid['population']

In [16]:
max_cases_per_capita = covid.new_cases_per_capita.max()
min_cases_per_capita = covid.new_cases_per_capita.min()
covid['normalized_cases'] = (covid['new_cases_per_capita'] - min_cases_per_capita) / (max_cases_per_capita - min_cases_per_capita)
covid

,country,year,week,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,hosp_patients,population,mov_avg_cases,new_cases_per_capita,normalized_cases
0,AUS,2020,4,4.0,4.0,NaN,0.0,0.155,NaN,25788217.0,4.000000e+00,1.551096e-07,0.000005
1,AUS,2020,5,5.0,8.0,NaN,0.0,0.194,NaN,25788217.0,6.000000e+00,2.326644e-07,0.000007
2,AUS,2020,6,12.0,3.0,NaN,0.0,0.465,NaN,25788217.0,5.000000e+00,1.938870e-07,0.000006
3,AUS,2020,7,15.0,0.0,NaN,0.0,0.582,NaN,25788217.0,3.750000e+00,1.454152e-07,0.000005
4,AUS,2020,8,15.0,0.0,NaN,0.0,0.582,NaN,25788217.0,3.000000e+00,1.163322e-07,0.000004
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3651,USA,2022,1,56345834.0,5064970.0,830299.0,11876.0,169249.873,108779.0,332915074.0,1.794629e+06,5.390651e-03,0.167444
3652,USA,2022,2,61703251.0,5614161.0,842288.0,13542.0,185342.316,142248.0,332915074.0,2.523969e+06,7.581421e-03,0.235493
3653,USA,2022,3,66610143.0,4951791.0,854838.0,14705.0,200081.487,150906.0,332915074.0,3.108541e+06,9.337339e-03,0.290035
3654,USA,2022,4,71802940.0,3561887.0,870600.0,16801.0,215679.450,144756.0,332915074.0,3.498425e+06,1.050846e-02,0.326413


In [18]:
covid.to_csv('covid_normalized_cases.csv')

,country,year,week,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,hosp_patients,population,mov_avg_cases,new_cases_per_capita,normalized_cases
0,AUS,2020,4,4.0,4.0,NaN,0.0,0.155,NaN,25788217.0,4.000000e+00,1.551096e-07,0.000005
1,AUS,2020,5,5.0,8.0,NaN,0.0,0.194,NaN,25788217.0,6.000000e+00,2.326644e-07,0.000007
2,AUS,2020,6,12.0,3.0,NaN,0.0,0.465,NaN,25788217.0,5.000000e+00,1.938870e-07,0.000006
3,AUS,2020,7,15.0,0.0,NaN,0.0,0.582,NaN,25788217.0,3.750000e+00,1.454152e-07,0.000005
4,AUS,2020,8,15.0,0.0,NaN,0.0,0.582,NaN,25788217.0,3.000000e+00,1.163322e-07,0.000004
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3651,USA,2022,1,56345834.0,5064970.0,830299.0,11876.0,169249.873,108779.0,332915074.0,1.794629e+06,5.390651e-03,0.167444
3652,USA,2022,2,61703251.0,5614161.0,842288.0,13542.0,185342.316,142248.0,332915074.0,2.523969e+06,7.581421e-03,0.235493
3653,USA,2022,3,66610143.0,4951791.0,854838.0,14705.0,200081.487,150906.0,332915074.0,3.108541e+06,9.337339e-03,0.290035
3654,USA,2022,4,71802940.0,3561887.0,870600.0,16801.0,215679.450,144756.0,332915074.0,3.498425e+06,1.050846e-02,0.326413
